In [1]:
import urllib.request, urllib.parse, urllib.error
import ssl

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup
import lxml

Solving environment: done

# All requested packages already installed.



In [2]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:
url = ('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
html = urllib.request.urlopen(url, context=ctx).read()
soup = BeautifulSoup(html, 'lxml')

In [4]:
columns_names = []

headers = soup.tbody.tr.text.split('\n')

for name in headers:
    if len(name) > 1:
        columns_names.append(name)

In [5]:
neighborhoods = pd.DataFrame(columns=columns_names)

tab = soup.tbody.text.split('\n\n\n')[1:]

for n in zip(tab):
    postcode = str(n).split('\\n')[0][2:]
    borough = str(n).split('\\n')[1]
    neighbourhood = str(n).split('\\n')[2][:-3]

    if borough == 'Not assigned': continue
    if neighbourhood == 'Not assigned':
        neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighbourhood': borough,
                                          'Postcode': postcode}, ignore_index=True)
    else:
        neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighbourhood': neighbourhood,
                                          'Postcode': postcode}, ignore_index=True)      


In [6]:
neighborhoods['Postcode'].describe()

count     210
unique    103
top       M9V
freq        8
Name: Postcode, dtype: object

In [7]:
neighborhoods = neighborhoods.groupby('Postcode').agg({'Borough':'first',  'Neighbourhood': ', '.join }).reset_index()

In [8]:
neighborhoods.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
neighborhoods.shape

(103, 3)

End of the first task.
The BeautifulSoup4, pandas, urllib, ssl and lxml package have been used. 